In [1]:
!pip install --quiet geocoder

In [2]:
import pandas as pd

DEATHS = '/kaggle/input/mountain-climbing-accidents-dataset/deaths_on_eight-thousanders.csv'

df = pd.read_csv(filepath_or_buffer=DEATHS, parse_dates=['Date'])
df.head()

,Date,Name,Nationality,Cause of death,Mountain
0,2023-07-27,Muhammad Hassan,Pakistan,Unknown,K2
1,2022-07-22,Matthew Eakin,Australia,Fall,K2
2,2022-07-22,Richard Cartier,Canada,Fall,K2
3,2022-07-21,Ali Akbar Sakhi,Afghanistan,"Unknown, suspected altitude sickness",K2
4,2021-07-25,Rick Allen,United Kingdom,Avalanche,K2


In [3]:
from arrow import now
from geocoder import arcgis

time_start = now()
mountain_df = pd.DataFrame.from_dict(orient='index', data={mountain: arcgis(location=mountain).latlng for mountain in df['Mountain'].unique().tolist()}, ).reset_index().merge(right=df['Mountain'].value_counts().to_frame().reset_index(), left_on='index', right_on='Mountain', how='inner').drop(columns=['Mountain'])
mountain_df.columns = ['Mountain', 'latitude', 'longitude', 'count']
print('done in {}'.format(now() - time_start))

done in 0:00:11.939369


In [4]:
from plotly.express import scatter_mapbox
scatter_mapbox(data_frame=mountain_df, lat='latitude', lon='longitude', size='count', hover_name='Mountain', mapbox_style='open-street-map', zoom=3, height=900)

In [5]:
from plotly.express import histogram
for x in ['Date', 'Mountain']:
    histogram(data_frame=df, x=x).show()

In [6]:
for column in ['Nationality', 'Cause of death']:
    histogram(data_frame=df, y=column, height=1500).show()

In [7]:
from plotly.express import strip
strip(data_frame=df, y='Nationality', x='Mountain', hover_name='Name', hover_data=['Date'], height=1500, stripmode='overlay')